In [98]:
from bs4 import BeautifulSoup
import requests, sys
import pandas as pd
import os
import re
import time

from threading import get_ident
from multiprocessing import cpu_count
import asyncio
from concurrent.futures import ThreadPoolExecutor as PoolExecuter

from joblib.externals.loky import set_loky_pickler
from joblib import parallel_backend, Parallel, delayed, wrap_non_picklable_objects

def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0
    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    return out

r = requests.get("http://everynoise.com/engenremap.html")

In [110]:
import re
import sys
import requests
from bs4 import BeautifulSoup
from requests.exceptions import RequestException
from multiprocessing import cpu_count

def fetch_artists(soup):
    all_artist_divs = set(soup.find_all("div", "genre scanme"))
    artist_list = []
    art_weights = []

    for artist in all_artist_divs:
        weight = artist['style'].split()[-1].replace('%', '')
        artist_name = artist.text.strip().replace("»", "")

        if not artist_name.isspace():
            art_weights.append(weight)
            artist_list.append(artist_name)

    return artist_list, art_weights


def fetch_similar_and_opposite_genres(soup, subgenre_name):
    all_genres_related = set(soup.find_all("div", "genre"))
    all_artist_divs = set(soup.find_all("div", "genre scanme"))
    all_genres_related = all_genres_related - all_artist_divs

    similar_genres = []
    opposite_genres = []
    sim_weights = []
    opp_weights = []

    for other_genre in all_genres_related:
        weight = other_genre['style'].split()[-1].replace('%', '')

        if 'nearby' in other_genre['id']:
            genre = other_genre.text.strip().replace("»", '')

            if genre != subgenre_name:
                sim_weights.append(weight)
                similar_genres.append(genre)

        elif 'mirror' in other_genre['id']:
            opp_weights.append(weight)
            opposite_genres.append(other_genre.text.strip().replace("»", ''))

    return similar_genres, sim_weights, opposite_genres, opp_weights


def genre_scraper(subgenre_subset):
    # Metadata
    subgenre_list_artist = []
    subgenre_list_opp = []
    subgenre_list_sim = []
    subgenre_list = []
    playlists = []
    sim_weights = []
    opp_weights = []
    artist_weights = []

    split_genrecount = 0

    # Maximum number of allowed retries
    max_attempts = 5

    # Looping over subgenres to get metadata
    for genreDiv in subgenre_subset:
        subgenre = re.sub("[:'+»&\s-]", '', genreDiv)
        subgenre_name = re.sub("[:+»\s]", ' ', genreDiv).strip()
        subgenre_list.append(subgenre_name)

        # Printing Genre Name
        sys.stdout.flush()
        print('Pulling genre # {} - Subgenre: {}\n'.format(split_genrecount, subgenre_name))
        sys.stdout.flush()

        # Fetch the subgenre
        subgenre_page = 'http://everynoise.com/engenremap-' + subgenre + '.html'
        skip_subgenre = False

        # Retry the request with a maximum number of attempts
        for attempt in range(max_attempts):
            try:
                r2 = requests.get(subgenre_page, timeout=10)
                break
            except RequestException as e:
                if attempt < max_attempts - 1:
                    print(f"Request {subgenre_page} failed with error '{e}', retrying... (attempt {attempt+1})")
                else:
                    print(f"Max retries exceeded for {subgenre_page}. Skipping...")
                    skip_subgenre = True
                    break

        if skip_subgenre:
            continue

        soup2 = BeautifulSoup(r2.text, 'html.parser')

        # Get the Spotify playlist link
        spotify_link = soup2.find_all('a', text='playlist')
        playlists.append(spotify_link[0]['href'] if spotify_link else None)

        # Fetch artists associated with the subgenre
        artist_list, art_weights = fetch_artists(soup2)
        subgenre_list_artist.append(artist_list)
        artist_weights.append(art_weights)

        # Get similar / opposite genres
        sim_genres, sim_gen_weights, opp_genres, opp_gen_weights = fetch_similar_and_opposite_genres(soup2, subgenre_name)
        subgenre_list_sim.append(sim_genres)
        subgenre_list_opp.append(opp_genres)
        sim_weights.append(sim_gen_weights)
        opp_weights.append(opp_gen_weights)

        split_genrecount += 1

    return [subgenre_list, playlists, subgenre_list_sim, sim_weights, subgenre_list_opp, opp_weights, subgenre_list_artist, artist_weights]

def parallel_wrapper(allGenreDivs, n_jobs=cpu_count()):
    subgenre_subsets = chunkIt(allGenreDivs, n_jobs)
    results = Parallel(n_jobs=n_jobs)(delayed(genre_scraper)(subset) for subset in subgenre_subsets)
    res = list(map(list, zip(*results)))
    res_2 = []
    
    for i in res:
        q = []
        for j in i:
            q += j
        res_2.append(q)

    return res_2

In [112]:
soup = BeautifulSoup(r.text,"html.parser")
allGenreDivs = soup.find_all("div", "genre scanme")
agd = []
for i in allGenreDivs:
    agd.append(i.text)
t1 = time.time()
music_data = parallel_wrapper(agd)
t2 = time.time()
print('Time Taken to Gather Data: {:.02f}s'.format(t2-t1))

Pulling genre # 0 - Subgenre: musica feirense

Pulling genre # 0 - Subgenre: comedy

Pulling genre # 0 - Subgenre: rap salvadoreno

Pulling genre # 0 - Subgenre: dangdut

Pulling genre # 0 - Subgenre: galician rock

Pulling genre # 0 - Subgenre: pop

Pulling genre # 0 - Subgenre: south sudanese pop

Pulling genre # 0 - Subgenre: tallava

Pulling genre # 1 - Subgenre: fort worth indie

Pulling genre # 1 - Subgenre: bisaya worship

Pulling genre # 1 - Subgenre: garifuna folk

Pulling genre # 1 - Subgenre: street punk espanol

Pulling genre # 1 - Subgenre: german boom bap

Pulling genre # 1 - Subgenre: chicago house

Pulling genre # 1 - Subgenre: rap

Pulling genre # 1 - Subgenre: trancecore

Pulling genre # 2 - Subgenre: meenawati

Pulling genre # 2 - Subgenre: zim gospel

Pulling genre # 2 - Subgenre: jazz puertorriqueno

Pulling genre # 2 - Subgenre: congolese gospel

Pulling genre # 2 - Subgenre: pianissimo

Pulling genre # 2 - Subgenre: shabad

Pulling genre # 2 - Subgenre: rock

Pul

Pulling genre # 23 - Subgenre: canadian black metal

Pulling genre # 23 - Subgenre: danzon

Pulling genre # 23 - Subgenre: vincy soca

Pulling genre # 23 - Subgenre: cumbia peruana

Pulling genre # 21 - Subgenre: album rock

Pulling genre # 24 - Subgenre: ugandan hip hop

Pulling genre # 23 - Subgenre: french tech house

Pulling genre # 23 - Subgenre: czech punk

Pulling genre # 24 - Subgenre: polyphonies corses

Pulling genre # 24 - Subgenre: beach music

Pulling genre # 24 - Subgenre: rock chapin

Pulling genre # 24 - Subgenre: intelligent dance music

Pulling genre # 25 - Subgenre: candombe

Pulling genre # 24 - Subgenre: lesen

Pulling genre # 22 - Subgenre: contemporary country

Pulling genre # 24 - Subgenre: touhou

Pulling genre # 25 - Subgenre: samba gospel

Pulling genre # 25 - Subgenre: arkansas indie

Pulling genre # 25 - Subgenre: trival

Pulling genre # 25 - Subgenre: ocean

Pulling genre # 26 - Subgenre: finnish electronic

Pulling genre # 25 - Subgenre: russian edm

Pull

Pulling genre # 46 - Subgenre: trap maroc

Pulling genre # 45 - Subgenre: drone metal

Pulling genre # 45 - Subgenre: arab electronic

Pulling genre # 46 - Subgenre: serbian electronic

Pulling genre # 47 - Subgenre: streichquartett

Pulling genre # 44 - Subgenre: folklore argentino

Pulling genre # 47 - Subgenre: lao pop

Pulling genre # 42 - Subgenre: sertanejo universitario

Pulling genre # 47 - Subgenre: rhode island indie

Pulling genre # 46 - Subgenre: dream trance

Pulling genre # 46 - Subgenre: drain

Pulling genre # 47 - Subgenre: sheilat

Pulling genre # 48 - Subgenre: folclore castilla y leon

Pulling genre # 45 - Subgenre: boston rock

Pulling genre # 48 - Subgenre: indie pop rock

Pulling genre # 43 - Subgenre: alternative rock

Pulling genre # 48 - Subgenre: pittsburgh rock

Pulling genre # 47 - Subgenre: christian afrobeat

Pulling genre # 47 - Subgenre: classic french pop

Pulling genre # 48 - Subgenre: berlin minimal techno

Pulling genre # 49 - Subgenre: grim death me

Pulling genre # 65 - Subgenre: musica infantil

Pulling genre # 62 - Subgenre: gangster rap

Pulling genre # 69 - Subgenre: japanese techno

Pulling genre # 70 - Subgenre: danish folk

Pulling genre # 68 - Subgenre: british modern classical

Pulling genre # 69 - Subgenre: contemporary folk

Pulling genre # 69 - Subgenre: korean indie folk

Pulling genre # 68 - Subgenre: rap boricua

Pulling genre # 66 - Subgenre: gujarati pop

Pulling genre # 63 - Subgenre: latin alternative

Pulling genre # 71 - Subgenre: oromo pop

Pulling genre # 69 - Subgenre: hokkaido indie

Pulling genre # 70 - Subgenre: guam indie

Pulling genre # 70 - Subgenre: worcester ma indie

Pulling genre # 70 - Subgenre: west australian hip hop

Pulling genre # 69 - Subgenre: rock independant francais

Pulling genre # 67 - Subgenre: rock urbano mexicano

Pulling genre # 64 - Subgenre: ranchera

Pulling genre # 70 - Subgenre: modern hardcore

Pulling genre # 72 - Subgenre: anime game

Pulling genre # 71 - Subgenre: sesoth

Pulling genre # 90 - Subgenre: indonesian blues

Pulling genre # 83 - Subgenre: latin rock

Pulling genre # 91 - Subgenre: pinoy praise

Pulling genre # 93 - Subgenre: luxembourgian hip hop

Pulling genre # 89 - Subgenre: pop nacional antigas

Pulling genre # 92 - Subgenre: serbian indie

Pulling genre # 91 - Subgenre: drift

Pulling genre # 91 - Subgenre: ottawa indie

Pulling genre # 87 - Subgenre: italian underground hip hop

Pulling genre # 84 - Subgenre: rap metal

Pulling genre # 92 - Subgenre: malayalam indie

Pulling genre # 93 - Subgenre: hip hop galsen

Pulling genre # 94 - Subgenre: romanian contemporary classical

Pulling genre # 90 - Subgenre: parody

Pulling genre # 92 - Subgenre: kingston on indie

Pulling genre # 92 - Subgenre: french garage rock

Pulling genre # 88 - Subgenre: toronto indie

Pulling genre # 85 - Subgenre: modern alternative rock

Pulling genre # 95 - Subgenre: tuna

Pulling genre # 94 - Subgenre: rap abc paulista

Pulling genre # 93 - Subgenre: undergr

...

Pulling genre # 616 - Subgenre: smooth jazz

Pulling genre # 647 - Subgenre: swedish hard rock

Pulling genre # 689 - Subgenre: musica mogiana

Pulling genre # 666 - Subgenre: indonesian indie pop

Pulling genre # 674 - Subgenre: deep discofox

Pulling genre # 688 - Subgenre: dutch underground hip hop

Pulling genre # 651 - Subgenre: fingerstyle

Pulling genre # 644 - Subgenre: hungarian classical performance

Pulling genre # 617 - Subgenre: jazz funk

Pulling genre # 667 - Subgenre: bleep techno

Pulling genre # 648 - Subgenre: washington indie

Pulling genre # 675 - Subgenre: thai instrumental

Pulling genre # 652 - Subgenre: memphis blues

Pulling genre # 645 - Subgenre: marimba de guatemala

Pulling genre # 689 - Subgenre: russian power metal

Pulling genre # 690 - Subgenre: draaiorgel

Pulling genre # 618 - Subgenre: trova

Pulling genre # 668 - Subgenre: bogor indie

Pulling genre # 676 - Subgenre: szanty

Pulling genre # 653 - Subgenre: experimental classical

Pulling genre # 64

Pulling genre # 709 - Subgenre: karen pop

Pulling genre # 710 - Subgenre: peruvian experimental

Pulling genre # 665 - Subgenre: music hall

Pulling genre # 668 - Subgenre: chihuahua indie

Pulling genre # 637 - Subgenre: albanian hip hop

Pulling genre # 697 - Subgenre: estonian indie

Pulling genre # 688 - Subgenre: danish post-punk

Pulling genre # 711 - Subgenre: vintage swoon

Pulling genre # 710 - Subgenre: jazz dominicano

Pulling genre # 672 - Subgenre: sinhala indie

Pulling genre # 666 - Subgenre: afrikaans hip hop

Pulling genre # 669 - Subgenre: christian metalcore

Pulling genre # 638 - Subgenre: nashville sound

Pulling genre # 698 - Subgenre: indian metal

Pulling genre # 689 - Subgenre: spanish folk rock

Pulling genre # 712 - Subgenre: wandelweiser

Pulling genre # 711 - Subgenre: grunge revival

Pulling genre # 673 - Subgenre: magyar alternative

Pulling genre # 667 - Subgenre: thall

Pulling genre # 639 - Subgenre: rap genovese

Pulling genre # 670 - Subgenre: melod

Pulling genre # 730 - Subgenre: russian black metal

Pulling genre # 688 - Subgenre: gospel r&b

Pulling genre # 732 - Subgenre: chinese opera

Pulling genre # 659 - Subgenre: chicano rap

Pulling genre # 719 - Subgenre: australian hardcore

Pulling genre # 694 - Subgenre: atlanta bass

Pulling genre # 709 - Subgenre: dutch prog

Pulling genre # 731 - Subgenre: classic ukrainian pop

Pulling genre # 733 - Subgenre: deep breakcore

Pulling genre # 660 - Subgenre: finnish hip hop

Pulling genre # 687 - Subgenre: southampton indie

Pulling genre # 689 - Subgenre: taiwan hip hop

Pulling genre # 720 - Subgenre: sean-nos singing

Pulling genre # 695 - Subgenre: futuristic swag

Pulling genre # 710 - Subgenre: musica istmena

Pulling genre # 661 - Subgenre: disco house

Pulling genre # 734 - Subgenre: papuan traditional

Pulling genre # 688 - Subgenre: dub punk

Pulling genre # 732 - Subgenre: pops orchestra

Pulling genre # 690 - Subgenre: christian trap

Pulling genre # 721 - Subgenre: sou

Pulling genre # 680 - Subgenre: new jersey rap

Pulling genre # 754 - Subgenre: himene tarava

Pulling genre # 707 - Subgenre: norwegian americana

Pulling genre # 730 - Subgenre: persian melodic rap

Pulling genre # 710 - Subgenre: chiptune

Pulling genre # 715 - Subgenre: straight edge

Pulling genre # 741 - Subgenre: canadian death metal

Pulling genre # 752 - Subgenre: chinese experimental

Pulling genre # 681 - Subgenre: covertronica

Pulling genre # 755 - Subgenre: yunnan traditional

Pulling genre # 708 - Subgenre: uk beatdown

Pulling genre # 731 - Subgenre: tampa indie

Pulling genre # 711 - Subgenre: afrikaans

Pulling genre # 742 - Subgenre: hampton roads indie

Pulling genre # 682 - Subgenre: british singer-songwriter

Pulling genre # 756 - Subgenre: classical piano quartet

Pulling genre # 716 - Subgenre: vgm remix

Pulling genre # 709 - Subgenre: czsk reggae

Pulling genre # 732 - Subgenre: musica ayacuchana

Pulling genre # 753 - Subgenre: hungarian classical piano

Pull

Pulling genre # 733 - Subgenre: canadian experimental

Pulling genre # 706 - Subgenre: small room

Pulling genre # 758 - Subgenre: chinese indie rock

Pulling genre # 741 - Subgenre: collage pop

Pulling genre # 737 - Subgenre: future rock

Pulling genre # 734 - Subgenre: rap galego

Pulling genre # 707 - Subgenre: philly indie

Pulling genre # 759 - Subgenre: cyber metal

Pulling genre # 738 - Subgenre: ethereal wave

Pulling genre # 742 - Subgenre: ontario indie

Pulling genre # 735 - Subgenre: fijian pop

Pulling genre # 708 - Subgenre: japanese emo

Pulling genre # 760 - Subgenre: modern funk

Pulling genre # 743 - Subgenre: russian chanson

Pulling genre # 739 - Subgenre: northern irish indie

Pulling genre # 736 - Subgenre: deep deep house

Pulling genre # 709 - Subgenre: gothic metal

Pulling genre # 761 - Subgenre: smooth soul

Pulling genre # 744 - Subgenre: kurdish hip hop

Pulling genre # 740 - Subgenre: spanish folk metal

Pulling genre # 710 - Subgenre: indonesian rock

Pu

In [5]:
import sys
import re
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd

class GenreProcessingError(Exception):
    def __init__(self, message, genre, *args, **kwargs):
        super(GenreProcessingError, self).__init__(message, *args, **kwargs)
        self.genre = genre

def fetch_genre_page(subgenre):
    try:
        genre_page_url = f"http://everynoise.com/engenremap-{subgenre}.html"
        response = requests.get(genre_page_url)
        return BeautifulSoup(response.text, "html.parser")
    except requests.RequestException as request_err:
        raise GenreProcessingError("Error fetching genre page.", subgenre) from request_err

def process_genre(genre_div):
    genre = re.sub("[:'+»&\s-]", '', genre_div.text)
    soup2 = fetch_genre_page(genre)

    spotify_link = soup2.find_all("a", text='playlist')
    playlist = spotify_link[0]['href'] if len(spotify_link) > 0 else None

    all_artist_divs = set(soup2.find_all("div", "genre scanme"))
    all_genres_related = set(soup2.find_all("div", "genre")) - all_artist_divs

    artist_weights = [
        artist['style'].split()[-1].replace('%', '') for artist in all_artist_divs
    ]
    artists = [
        artist.text.strip().replace("»", "") for artist in all_artist_divs
        if not artist.text.strip().replace("»", "").isspace()
    ]

    sim_weights = [
        weight['style'].split()[-1].replace('%', '')
        for weight in all_genres_related if 'nearby' in weight['id']
    ]
    opp_weights = [
        weight['style'].split()[-1].replace('%', '')
        for weight in all_genres_related if 'mirror' in weight['id']
    ]

    sim_genres = [
        other_genre.text.strip().replace("»", "")
        for other_genre in all_genres_related if 'nearby' in other_genre['id']
    ]
    opp_genres = [
        other_genre.text.strip().replace("»", "")
        for other_genre in all_genres_related if 'mirror' in other_genre['id']
    ]

    return {
        'genre': genre,
        'playlist': playlist,
        'artist_weights': artist_weights,
        'artists': artists,
        'sim_weights': sim_weights,
        'opp_weights': opp_weights,
        'sim_genres': sim_genres,
        'opp_genres': opp_genres,
    }
    
def process_genres_and_update_csv(filename):
    df = pd.read_csv(filename)

    for idx, row in df.iterrows():
        if pd.isnull(row['SPOTIFY_URL']):
            genre_div = BeautifulSoup(f'<div>{row["GENRE"]}</div>', "html.parser")
            try:
                result = process_genre(genre_div)
                print(f"Processed genre #{idx}: {result['genre']}")

                # Update the corresponding row in the DataFrame
                df.at[idx, 'SIM_GENRES'] = ', '.join(result['sim_genres'])
                df.at[idx, 'SIM_WEIGHTS'] = ', '.join(result['sim_weights'])
                df.at[idx, 'OPP_GENRES'] = ', '.join(result['opp_genres'])
                df.at[idx, 'OPP_WEIGHTS'] = ', '.join(result['opp_weights'])
                df.at[idx, 'REL_ARTISTS'] = ', '.join(result['artists'])
                df.at[idx, 'ARTIST_WEIGHTS'] = ', '.join(result['artist_weights'])
                df.at[idx, 'SPOTIFY_URL'] = result['playlist']
            except GenreProcessingError as gpe:
                print(f"Error processing genre #{idx} ({gpe.genre}): {gpe}")

    # Save the updated DataFrame back to the CSV file
    df.to_csv(filename, index=False)

filename = "all_genres1.csv"
process_genres_and_update_csv(filename)

response = requests.get("http://everynoise.com/engenremap.html")
soup = BeautifulSoup(response.text, "html.parser")
all_genre_divs = soup.find_all("div", "genre scanme")

results = []

for index, genre_div in enumerate(all_genre_divs):
    genre = re.sub("[:'+»&\s-]", '', genre_div.text)

    if genre in genres_to_process:
        try:
            result = process_genre(genre_div)
            results.append(result)
            print(f"Processed genre #{index}: {result['genre']}")
        except GenreProcessingError as gpe:
            print(f"Error processing genre #{index} ({gpe.genre}): {gpe}")


Processed genre #208: hardcorehiphop
Processed genre #211: popvenezolano
Processed genre #212: frenchpop
Processed genre #215: popreggaeton
Processed genre #216: classicsoul
Processed genre #219: dutchhouse
Processed genre #220: orchestralsoundtrack
Processed genre #222: popsoul
Processed genre #224: outlawcountry
Processed genre #225: braziliangospel
Processed genre #226: calirap
Processed genre #434: viralrap
Processed genre #438: indonesiansingersongwriter
Processed genre #441: ambientpop
Processed genre #442: ukalternativepop
Processed genre #443: kentuckyhiphop
Processed genre #448: brazilianreggae
Processed genre #449: czskhiphop
Processed genre #452: latinworship
Processed genre #454: jazzpop
Processed genre #458: pagodebaiano
Processed genre #465: mexicanindie
Processed genre #466: earlymusic
Processed genre #467: germandrill
Processed genre #468: frenchrock
Processed genre #469: elpasoindie
Processed genre #470: countryrap
Processed genre #477: drillespanol
Processed genre #47

Processed genre #1265: afghanpop
Processed genre #1266: russianpostpunk
Processed genre #1267: boombapbrasileiro
Processed genre #1269: chicagosoul
Processed genre #1271: canzonenapoletana
Processed genre #1275: ghanaianpop
Processed genre #1276: classiccantopop
Processed genre #1280: psychedelictrance
Processed genre #1287: swedishindierock
Processed genre #1290: russianpunk
Processed genre #1296: cancioninfantillatinoamericana
Processed genre #1297: riotgrrrl
Processed genre #1300: australianalternativepop
Processed genre #1302: celticpunk
Processed genre #1305: ukrainianrock
Processed genre #1307: mexicanclassicrock
Processed genre #1312: russiandrill
Processed genre #1315: sludgemetal
Processed genre #1318: neueneuedeutschewelle
Processed genre #1321: vikingmetal
Processed genre #1324: mariachicristiano
Processed genre #1325: canadianccm
Processed genre #1332: classicrussianpop
Processed genre #1334: anthememo
Processed genre #1336: folkrockitaliano
Processed genre #1337: kiwirock


Processed genre #2084: frenchdeathmetal
Processed genre #2087: polishpunk
Processed genre #2088: indianrock
Processed genre #2092: deepchill
Processed genre #2095: musicaperbambini
Processed genre #2097: jreggae
Processed genre #2098: balkanbeats
Processed genre #2100: bostonmetal
Processed genre #2101: ocindie
Processed genre #2102: worldfusion
Processed genre #2108: grooveroom
Processed genre #2109: acidhouse
Processed genre #2110: rockquebecois
Processed genre #2112: malianblues
Processed genre #2117: southafricantrap
Processed genre #2118: estonianhiphop
Processed genre #2119: fremantleindie
Processed genre #2120: neotraditionalbluegrass
Processed genre #2122: japanesedancepop
Processed genre #2124: alaskaindie
Processed genre #2126: salsachoke
Processed genre #2129: nzhiphop
Processed genre #2130: bongoflava
Processed genre #2131: deeptechhouse
Processed genre #2133: musicaalagoana
Processed genre #2134: newitalodisco
Processed genre #2135: vlaamsekinderliedje
Processed genre #213

Processed genre #2923: danishjazz
Processed genre #2924: rapundergroundcolombiano
Processed genre #2926: autonomousblackmetal
Processed genre #2929: kannadapop
Processed genre #2931: symphonicdeathmetal
Processed genre #2932: nordnorskmusikk
Processed genre #2938: classicsinhalapop
Processed genre #2946: cypriotpop
Processed genre #2947: vietnamesebolero
Processed genre #2948: mongolianalternative
Processed genre #2950: indonesianindiepop
Processed genre #2951: bleeptechno
Processed genre #2952: bogorindie
Processed genre #2954: wufam
Processed genre #2955: bassmusic
Processed genre #2959: munichelectronic
Processed genre #2964: victorianbritain
Processed genre #2967: horrorsynth
Processed genre #2968: indienapoletano
Processed genre #2970: coveracustico
Processed genre #2972: danishpostpunk
Processed genre #2973: spanishfolkrock
Processed genre #2976: spanishreggae
Processed genre #2977: atmosphericdnb
Processed genre #2978: okcindie
Processed genre #2979: windsoronindie
Processed gen

Processed genre #3743: armenianfolk
Processed genre #3745: singaporeanhiphop
Processed genre #3746: indiefolkitaliano
Processed genre #3747: musicalombarda
Processed genre #3748: halifaxindie
Processed genre #3750: mississippiindie
Processed genre #3751: frenchblackmetal
Processed genre #3753: norwegianamericana
Processed genre #3754: ukbeatdown
Processed genre #3755: czskreggae
Processed genre #3756: aarhusindie
Processed genre #3757: deepbrazilianpop
Processed genre #3758: segamauricien
Processed genre #3759: histoirepourenfants
Processed genre #3761: tucsonindie
Processed genre #3762: nyroots
Processed genre #3763: musicaevangelicainstrumental
Processed genre #3764: deepdubstep
Processed genre #3765: northdakotaindie
Processed genre #3766: swedishprog
Processed genre #3767: ryukyuongaku
Processed genre #3770: flemishfolk
Processed genre #3771: garagepsych
Processed genre #3773: britishindustrial
Processed genre #3774: bahamianpop
Processed genre #3775: spanishrockabilly
Processed ge

Processed genre #4560: swedishexperimental
Processed genre #4561: czechalternativerap
Processed genre #4563: czskelectropop
Processed genre #4565: deeprb
Processed genre #4566: dessinanime
Processed genre #4568: japanesedeathmetal
Processed genre #4801: hawaiianindie
Processed genre #4802: organicambient
Processed genre #4806: geininsong
Processed genre #4810: mexicanpostrock
Processed genre #4811: sufichant
Processed genre #4812: faroeserock
Processed genre #4814: progressivethrash
Processed genre #4818: southcarolinametal
Processed genre #4819: australianpostpunk
Processed genre #5058: warmetal
Processed genre #5060: hammereddulcimer
Processed genre #5062: derryindie
Processed genre #5064: hotjazz
Processed genre #5066: musicamadeirense
Processed genre #5068: norwegiantechno
Processed genre #5069: newhampshireindie
Processed genre #5071: italianblackmetal
Processed genre #5075: namibianpop
Processed genre #5077: danishclassical
Processed genre #5085: bernindie
Processed genre #5087: 

Processed genre #5894: accordionband
Processed genre #5895: metaluruguayo
Processed genre #5897: balticblackmetal
Processed genre #5899: fifeanddrum
Processed genre #5900: koreanhardcore
Processed genre #5901: alternativehardcore
Processed genre #5905: classicalguitarquartet
Processed genre #5906: marathibalgeet
Processed genre #5908: balticchoir
Processed genre #5909: chaoticblackmetal
Processed genre #5910: austrianchoir
Processed genre #5911: yiddishfolk
Processed genre #5914: chineseblackmetal
Processed genre #5916: mexicanblackmetal
Processed genre #5923: maliantraditional
Processed genre #5927: musicasinfonica
Processed genre #5928: montanametal
Processed genre #5930: estonianjazz
Processed genre #5933: czechcontemporaryclassical
Processed genre #5936: iranianmetal
Processed genre #5943: uyghurfolk
Processed genre #5945: cosmicupliftingtrance
Processed genre #5946: metalbalear
Processed genre #5948: polishexperimental
Processed genre #5949: industrialnoise
Processed genre #5952: 

In [1]:
import pandas as pd
# [subgenre_list, playlists, subgenre_list_sim, sim_weights, subgenre_list_opp, opp_weights, subgenre_list_artist, artist_weights]
#df = pd.DataFrame({"GENRE":music_data[0], 
                  # "SIM_GENRES":music_data[2],
                  # "SIM_WEIGHTS":music_data[3],
                  # "OPP_GENRES":music_data[4],
                  # "OPP_WEIGHTS":music_data[5],
                  # "REL_ARTISTS":music_data[6], 
                  # "ARTIST_WEIGHTS":music_data[7],
                  # "SPOTIFY_URL":music_data[1]})
#df.to_csv('all_genres1.csv')
df = pd.read_csv("all_genres1.csv")
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6091 entries, 0 to 6092
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   GENRE           6091 non-null   object
 1   SIM_GENRES      6091 non-null   object
 2   SIM_WEIGHTS     6091 non-null   object
 3   OPP_GENRES      6091 non-null   object
 4   OPP_WEIGHTS     6091 non-null   object
 5   REL_ARTISTS     6091 non-null   object
 6   ARTIST_WEIGHTS  6091 non-null   object
 7   SPOTIFY_URL     6091 non-null   object
dtypes: object(8)
memory usage: 428.3+ KB


In [1]:
import spotipy
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyClientCredentials

sp_client_id = ''
sp_client_secret = ''
credentials = oauth2.SpotifyClientCredentials(client_id=sp_client_id, client_secret=sp_client_secret)
sp = spotipy.Spotify(client_credentials_manager=credentials)

features = ['popularity', 'danceability', 'energy', 'key', 'loudness',
                    'mode', 'speechiness', 'acousticness', 'instrumentalness',
                    'liveness', 'valence', 'tempo', 'time_signature']

In [5]:
import time
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import json

def get_track_ids_from_playlist(sp, playlist_id):

    results = sp.playlist_tracks(playlist_id)
    track_ids = [item['track']['id'] for item in results['items']]
    return track_ids


def get_track_ids_for_playlists(sp, playlists):
    track_ids_dict = {}

        # Load existing track IDs
    try:
        with open('track_ids.json', 'r') as f:
            track_ids_dict = json.load(f)
    except FileNotFoundError:
        pass
    # Define a function for processing a single playlist URL in a separate thread
    def process_playlist_url(playlist_url):
        nonlocal track_ids_dict

        if playlist_url is None:
            track_ids_dict[playlist_url] = None
        elif playlist_url not in track_ids_dict:
            playlist_id = playlist_url.split('playlist/')[1]

            success, retries = False, 0
            while not success and retries < 2:  # Attempt once and retry once
                try:
                    track_ids = get_track_ids_from_playlist(sp, playlist_id)
                    track_ids_dict[playlist_url] = track_ids
                    success = True
                except Exception as e:
                    sleep_time = 2 ** retries
                    time.sleep(sleep_time)
                    retries += 1

    # Process all playlist URLs concurrently
    with ThreadPoolExecutor() as executor:
        list(tqdm(executor.map(process_playlist_url, playlists), total=len(playlists), desc='Processing playlists'))

    # Save track IDs to a file
    with open('track_ids.json', 'w') as f:
        json.dump(track_ids_dict, f)

    return track_ids_dict

playlists = [url for url in df['SPOTIFY_URL'].tolist() if isinstance(url, str)]
playlist_track_ids = get_track_ids_for_playlists(sp, playlists)

Processing playlists: 100%|█████████████████| 6091/6091 [02:46<00:00, 36.65it/s]


In [8]:
from ratelimiter import RateLimiter
sp = spotipy.Spotify(client_credentials_manager=spotipy.oauth2.SpotifyClientCredentials(client_id = '', client_secret = ''))

def get_track_popularity(sp, track_ids):
    track_popularity = {}
    
    tracks = sp.tracks(track_ids)
    for track in tracks['tracks']:
        track_popularity[track['id']] = track['popularity']

    return track_popularity

def get_audio_features_for_tracks(sp, track_ids_dict):
    audio_features_dict = {}
        # Load existing audio features
    try:
        with open('audio_features.json', 'r') as f:
            audio_features_dict = json.load(f)
    except FileNotFoundError:
        pass
    
    rate_limiter = RateLimiter(max_calls=30, period=1)
    # Adjust the rate limit as needed

    def process_track_ids(track_ids):
        nonlocal audio_features_dict
        batch_size = 50
    
        for i in range(0, len(track_ids), batch_size):
            batch = track_ids[i:i + batch_size]
        
        # Divide the batch into two parts: unprocessed_batch and processed_batch
        unprocessed_batch = [tid for tid in batch if tid not in audio_features_dict]
        processed_batch = [tid for tid in batch if tid in audio_features_dict]
        
        if unprocessed_batch:
            success, retries = False, 0
            while not success and retries < 10:  # Attempt once and retry once
                with rate_limiter:
                    try:
                        features = sp.audio_features(unprocessed_batch)
                        popularity = get_track_popularity(sp, unprocessed_batch)
                        
                        for feature in features:
                            feature['popularity'] = popularity[feature['id']]
                            audio_features_dict[feature['id']] = feature
                        
                        success = True
                    except Exception as e:
                        sleep_time = 2 ** retries
                        time.sleep(sleep_time)
                        retries += 1
        
        # Update popularity for processed tracks
        if processed_batch:
            popularity = get_track_popularity(sp, processed_batch)
                
            for tid in processed_batch:
                audio_features_dict[tid]['popularity'] = popularity[tid]
                
    # Process all track IDs concurrently
    with ThreadPoolExecutor(max_workers=6) as executor:
        track_ids_list = [ids for ids in track_ids_dict.values() if ids is not None]
        all_track_ids = [tid for ids in track_ids_list for tid in ids]
        chunks = [all_track_ids[i:i+50] for i in range(0, len(all_track_ids), 50)]
        list(tqdm(executor.map(process_track_ids, chunks), total=len(chunks), desc='Processing audio features'))
    # Save audio features to a file
    with open('audio_features.json', 'w') as f:
        json.dump(audio_features_dict, f)

    return audio_features_dict

audio_features = get_audio_features_for_tracks(sp, playlist_track_ids)

Processing audio features:  26%|██▊        | 3069/11805 [05:25<15:25,  9.43it/s]

KeyboardInterrupt



In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# Create a dictionary with genres as keys and lists of track IDs as values
genre_track_ids = {}
for index, row in df.iterrows():
    genre = row['GENRE']
    playlist_url = row['SPOTIFY_URL']
    track_ids = playlist_track_ids.get(playlist_url, [])
    if track_ids is not None:
        if genre not in genre_track_ids:
            genre_track_ids[genre] = []
        genre_track_ids[genre].extend(track_ids)

# Calculate the average of each audio feature for the tracks in each genre
genre_audio_features_avg = {}
for genre, track_ids in tqdm(genre_track_ids.items(), desc='Processing genres'):
    features_list = [audio_features[tid] for tid in track_ids if tid in audio_features]
    if features_list:
        features_df = pd.DataFrame(features_list)
        features_mean = features_df.mean(numeric_only=True)
        genre_audio_features_avg[genre] = features_mean.to_dict()
    else:
        genre_audio_features_avg[genre] = None

In [19]:
genre_audio_features_avg

{'pop': {'danceability': 0.6620600000000002,
  'energy': 0.6511099999999999,
  'key': 5.12,
  'loudness': -5.726880000000001,
  'mode': 0.64,
  'speechiness': 0.07093500000000001,
  'acousticness': 0.1975984,
  'instrumentalness': 0.006247488700000001,
  'liveness': 0.17319500000000002,
  'valence': 0.512612,
  'tempo': 123.58468999999997,
  'duration_ms': 204923.68,
  'time_signature': 3.94},
 'rap': {'danceability': 0.7286400000000001,
  'energy': 0.63693,
  'key': 5.77,
  'loudness': -6.236810000000002,
  'mode': 0.56,
  'speechiness': 0.15241000000000002,
  'acousticness': 0.14459235000000004,
  'instrumentalness': 0.0012794415999999996,
  'liveness': 0.196954,
  'valence': 0.4643110000000001,
  'tempo': 114.62312999999999,
  'duration_ms': 223874.0,
  'time_signature': 3.96},
 'rock': {'danceability': 0.5032700000000001,
  'energy': 0.71774,
  'key': 4.68,
  'loudness': -7.786729999999999,
  'mode': 0.7,
  'speechiness': 0.058156,
  'acousticness': 0.12671333900000004,
  'instrume

In [15]:
import pandas as pd
import numpy as np

# (same code as before to create genre_track_ids and genre_audio_features_avg)

# Filter the original DataFrame based on genres with averaged out features
valid_genres = [genre for genre, features in genre_audio_features_avg.items() if features is not None]
filtered_df = df[df['GENRE'].isin(valid_genres)].copy()

# Add the average features to the filtered DataFrame
numeric_features = features_df.select_dtypes(include=np.number).columns
for feature_name in numeric_features:
    filtered_df[f'{feature_name}'] = filtered_df['GENRE'].apply(lambda genre: genre_audio_features_avg[genre][feature_name])

# Print the filtered DataFrame with average features
filtered_df.head()
filtered_df.to_csv("avg_genres.csv", index=False)
    

